In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from glob import glob
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes

In [2]:
data_directory = 'data/'
image_directory = 'data/Pre_train/'

In [3]:
size = 32
batch_size = 256
num_channels = 3

In [4]:
def read_label_files(csv_file):
    file = open(data_directory+csv_file, 'r')
    filepaths = []
    labels = []
    for i in file:
        filename, label = i.split(',')
        filepaths.append(image_directory+filename)
        labels.append(int(label))
    return filepaths, labels    

In [5]:
train_path, train_labels = read_label_files('train_set.csv')
vali_path, vali_labels = read_label_files('vali_set.csv')

In [6]:
train_path_tensor = ops.convert_to_tensor(train_path, dtype=dtypes.string)
vali_path_tensor = ops.convert_to_tensor(vali_path, dtype=dtypes.string)
train_labels_tensor = ops.convert_to_tensor(train_labels, dtype=dtypes.int32)
vali_labels_tensor = ops.convert_to_tensor(vali_labels, dtype=dtypes.int32)

In [7]:
train_input_queue = tf.train.slice_input_producer(
                                    [train_path_tensor, train_labels_tensor],
                                    shuffle=True)
vali_input_queue = tf.train.slice_input_producer(
                                    [vali_path_tensor, vali_labels_tensor],
                                    shuffle=True)

In [8]:
file_content = tf.read_file(train_input_queue[0])
train_image = tf.image.decode_jpeg(file_content, channels=num_channels)
train_label = train_input_queue[1]

file_content = tf.read_file(vali_input_queue[0])
vali_image = tf.image.decode_jpeg(file_content, channels=num_channels)
vali_label = vali_input_queue[1]

In [9]:
train_image.set_shape([size, size, num_channels])
vali_image.set_shape([size, size, num_channels])

train_batch = tf.train.batch(
                            [train_image, train_label],
                            batch_size= batch_size
                            )
vali_batch = tf.train.batch(
                            [vali_image, vali_label],
                            batch_size = batch_size
                            )

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    for i in range(5):
        print(sess.run(train_batch))
    coord.request_stop()
    coord.join(threads)
    